In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [2]:
class Mulitiheadattention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(Mulitiheadattention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.num_heads = num_heads
        self.d_model = d_model
        self.head_dim = d_model // num_heads
        self.qkv = nn.Linear(d_model, d_model * 3)
        self.out = nn.Linear(d_model, d_model)
    
    def scaled_dot_product_attention(self, q, k, v, mask=None):
        d_k = q.size(-1)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        return torch.matmul(attn, v), attn

    def split_heads(self, x):
        # x: (batch_size, seq_length, num_heads, head_dim)
        return x.transpose(1, 2)

    def combine_heads(self, x):
        batch_size, num_heads, seq_length, head_dim = x.size()
        x = x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        return x

    def forward(self, x, mask=None, kv=None):
        # x: query, kv: key/value（可选）
        batch_size, seq_length, d_model = x.size()
        if kv is None:
            # self-attention
            qkv = self.qkv(x).view(batch_size, seq_length, 3, self.num_heads, self.head_dim)
            q, k, v = qkv.unbind(dim=2)
        else:
            # cross-attention
            q = self.qkv(x).view(batch_size, seq_length, 3, self.num_heads, self.head_dim)[:, :, 0]
            k = self.qkv(kv).view(batch_size, kv.size(1), 3, self.num_heads, self.head_dim)[:, :, 1]
            v = self.qkv(kv).view(batch_size, kv.size(1), 3, self.num_heads, self.head_dim)[:, :, 2]
            # 恢复维度 (batch_size, seq_length, num_heads, head_dim)
            q = q.view(batch_size, seq_length, self.num_heads, self.head_dim)
            k = k.view(batch_size, kv.size(1), self.num_heads, self.head_dim)
            v = v.view(batch_size, kv.size(1), self.num_heads, self.head_dim)
        # transpose to (batch_size, num_heads, seq_length, head_dim)
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        attn_output, attn_weights = self.scaled_dot_product_attention(q, k, v, mask)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        return self.out(attn_output)

In [3]:
class positionwisefeedforward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(positionwisefeedforward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.linear3 = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.dropout(x)
        return x
class positionencoder(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(positionencoder, self).__init__()
        self.d_model = d_model
        self.pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

In [4]:
class encoderlayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.01):
        super(encoderlayer, self).__init__()
        self.self_attn = Mulitiheadattention(d_model, num_heads)
        self.ffn = positionwisefeedforward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, mask)
        x = self.norm1(x + self.dropout1(attn_output))
        ffn_output = self.ffn(x)
        return self.norm2(x + self.dropout2(ffn_output))

In [ ]:
class decoderlayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(decoderlayer, self).__init__()
        self.self_attn = Mulitiheadattention(d_model, num_heads)
        self.cross_attn = Mulitiheadattention(d_model, num_heads)
        self.ffn = positionwisefeedforward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
    
    def forward(self, x, memory, src_mask=None, tgt_mask=None):
        attn_output = self.self_attn(x, tgt_mask)
        x = self.norm1(x + self.dropout1(attn_output))
        cross_attn_output = self.cross_attn(x, src_mask, kv=memory)
        x = self.norm2(x + self.dropout2(cross_attn_output))
        ffn_output = self.ffn(x)
        return self.norm3(x + self.dropout3(ffn_output))

: 

In [ ]:
import matplotlib.pyplot as plt
class NodeTransformer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super(NodeTransformer, self).__init__()
        self.d_model = d_model
        self.position_encoder = positionencoder(d_model)
        self.encoder_layers = nn.ModuleList(
            [encoderlayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [decoderlayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )
        self.final_layer = nn.Linear(d_model, d_model)
    
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src = self.position_encoder(src)
        tgt = self.position_encoder(tgt)
        
        for layer in self.encoder_layers:
            src = layer(src, src_mask)
        
        for layer in self.decoder_layers:
            tgt = layer(tgt, src, src_mask, tgt_mask)
        
        return self.final_layer(tgt)
    
transformer = NodeTransformer(d_model=512, num_heads=32, d_ff=2048, num_layers=20)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=0.001)
src_data = torch.randint(0, 100, (32, 10, 512))  # Example source data
tgt_data = torch.randint(0, 100, (32, 10, 512))  # Example target data
loss_val = []
num_epoch = 40
for epoch in range(num_epoch):
    transformer.train()
    optimizer.zero_grad()
    
    output = transformer(src_data, tgt_data)
    target = torch.randint(0, 100, (32, 10))  # Example target labels
    
    loss_value = loss(output.view(-1, 512), target.view(-1))
    loss_val.append(loss_value.item())
    loss_value.backward()
    
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss_value.item()}')
        
plt.plot(range(1, num_epoch+1), loss_val, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

Epoch 1, Loss: 6.43387508392334
Epoch 2, Loss: 5.797059059143066
Epoch 3, Loss: 5.323826789855957
Epoch 4, Loss: 5.100070953369141
Epoch 5, Loss: 4.966413497924805
Epoch 6, Loss: 4.912578105926514
Epoch 7, Loss: 4.8155903816223145
Epoch 8, Loss: 4.790465831756592
Epoch 9, Loss: 4.756431579589844
Epoch 10, Loss: 4.8197526931762695
Epoch 11, Loss: 4.7569804191589355
Epoch 12, Loss: 4.744077682495117
Epoch 13, Loss: 4.746025085449219
Epoch 14, Loss: 4.735293388366699
Epoch 15, Loss: 4.752455711364746
Epoch 16, Loss: 4.7452802658081055
Epoch 17, Loss: 4.728141784667969
Epoch 18, Loss: 4.722260475158691
Epoch 19, Loss: 4.657217025756836
Epoch 20, Loss: 4.704895973205566
Epoch 21, Loss: 4.716107368469238
Epoch 22, Loss: 4.7105488777160645
Epoch 23, Loss: 4.692664623260498
Epoch 24, Loss: 4.705496788024902
Epoch 25, Loss: 4.6739912033081055
Epoch 26, Loss: 4.708690166473389
Epoch 27, Loss: 4.715512275695801
Epoch 28, Loss: 4.674624443054199
Epoch 29, Loss: 4.686723232269287
Epoch 30, Loss: 4.

In [ ]:
vad_data = torch.randint(0, 100, (32, 10, 512))  # Example validation data
transformer.eval()
with torch.no_grad():
    output = transformer(vad_data, vad_data)
    print(f'Validation output shape: {output.shape}') 
    vad_loss = loss(output.view(-1, 512), torch.randint(0, 100, (32, 10)).view(-1))
    print(f'Validation Loss: {vad_loss.item()}')